In [2]:
import pandas as pd
from pathlib import Path

In [3]:
path = Path('Data/')

In [4]:
df = pd.read_feather('Data/May2015_subset.fthr')

/home/gokul/anaconda3/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [10]:
df.head(3).T

,0,1,2
created_utc,1430438402,1430438411,1430438412
ups,1,12,1
subreddit_id,t5_2qh22,t5_2qh22,t5_2qh22
link_id,t3_34gb0n,t3_34gfze,t3_34b6gv
name,t1_cqug92d,t1_cqug99n,t1_cqug9aa
score_hidden,0,0,0
author_flair_css_class,None,MAL,MAL
author_flair_text,None,http://myanimelist.net/animelist/Link_of_Hyrule,http://myanimelist.net/profile/TenWoTsuke
subreddit,anime,anime,anime
id,cqug92d,cqug99n,cqug9aa


# What is t1, t3, and t5 ?
$t_i$ are 
![](images/reddit_API.png)

In [8]:
f"{df.memory_usage(deep=True).sum() / 1023.0**3} GB"

'5.128976683577038 GB'

! Ohh, let's reduce those memory footprint.

In [39]:
df.shape

(6195172, 22)

In [33]:
df['subreddit'] = df.subreddit.astype('category')
df['subreddit_id'] = df.subreddit_id.astype('category')
df['author_flair_css_class'] = df.author_flair_css_class.astype('category')
df['distinguished'] = df.distinguished.astype('category')
df['link_id'] = df.link_id.astype('category')

df.drop(['archived'], inplace=True) # Only one value i.e. 0
df = df[~(df.body == '[deleted]')]  # remove [deleted] comments

In [43]:
df.shape

(5703706, 22)

In [37]:
df.removal_reason.unique()

array([None, 'legal'], dtype=object)

In [34]:
f"{df.memory_usage(deep=True).sum() / 1023.0**3} GB"

'3.734180583385416 GB'

In [11]:
# No comments in a subreddits
df.subreddit.value_counts()

leagueoflegends      1223184
funny                 745916
news                  548287
todayilearned         519910
worldnews             490354
movies                376601
gaming                361265
politics              244927
explainlikeimfive     223148
anime                 221118
gameofthrones         191006
motorcycles            89721
science                89413
conspiracy             78349
tf2                    76549
sex                    74912
guns                   61234
comicbooks             60303
pokemon                59730
food                   55231
starcraft              53797
askscience             53017
mylittlepony           44119
space                  42726
skyrim                 40568
circlejerk             30134
facepalm               22562
community              20635
harrypotter            19998
progresspics           13385
drunk                  13027
startrek               12275
thewalkingdead         10357
offbeat                10035
lifehacks     

In [13]:
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
df.

count    6.195172e+06
mean     7.942966e+00
std      6.334749e+01
min     -1.428000e+03
25%      1.000000e+00
50%      1.000000e+00
75%      3.000000e+00
max      5.642000e+03
Name: score, dtype: float64